# 2. Niveles Lingüísticos II

<a target="_blank" href="https://colab.research.google.com/github/umoqnier/cl-2026-2-lab/blob/main/notebooks/2_niveles_linguisticos_II.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Objetivos

- Comparar enfoques basados en reglas y estadísticos para el análisis morfológico
- Los alumnæs comprenderán la importancia de las etiquetas *POS* en tareas de _NLP_
- Implementar un modelo de etiquetado automático
    - Usando modelos discriminativos _HMMs_
    - Usando modelos condicionales _CRFs_
    - Contrastar ambos enfoques para generación automática de secuencias de etiquetas

In [ ]:
# !uv add nltk scikit-learn sklearn-crfsuite <- Local con uv
!pip install nltk scikit-learn sklearn-crfsuite

In [ ]:
import re
from collections import defaultdict

import matplotlib.pyplot as plt
import nltk
import pandas as pd
import requests as r
from nltk.corpus import cess_esp

from rich import print as rprint

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF

## Morfología

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/29/Flexi%C3%B3nGato-svg.svg/800px-Flexi%C3%B3nGato-svg.svg.png" width=200></center>

> De <a href="//commons.wikimedia.org/wiki/User:KES47" class="mw-redirect" title="User:KES47">KES47</a> - <a href="//commons.wikimedia.org/wiki/File:Flexi%C3%B3nGato.png" title="File:FlexiónGato.png">File:FlexiónGato.png</a> y <a href="//commons.wikimedia.org/wiki/File:Nuvola_apps_package_toys_svg.svg" title="File:Nuvola apps package toys svg.svg">File:Nuvola apps package toys svg.svg</a>, <a href="http://www.gnu.org/licenses/lgpl.html" title="GNU Lesser General Public License">LGPL</a>, <a href="https://commons.wikimedia.org/w/index.php?curid=27305101">Enlace</a>

El análisis morfológico es la determinación de las partes que componen la palabra y su representación lingüística, es una especie de etiquetado

Los elementos morfológicos son analizados para:

- Determinar la función morfológica de las palabras
- Hacer filtrado y pre-procesamiento de texto

### Análisis morfológico basado en reglas

Recordemos que podemos hacer un analizador morfológico haciendo uso de un transductor que vaya leyendo y haciendo transformaciones en una cadena. Formalmente:

* $Q = \{q_0, \ldots, q_T\}$ conjunto finito de estados.
* $\Sigma$ es un alfabeto de entrada.
* $q_0 \in Q$ es el estado inicial.
* $F \subseteq Q$ es el conjunto de estados finales.

Un transductor es una 6-tupla $T = (Q, \Sigma, \Delta, q_0, F, \sigma)$ tal que

* $\Delta$ es un alfabeto de salida teminal
* $\Sigma$ es un alfabeto de entrada no terminal
* $\sigma: Q \times \Sigma \times \Delta \longrightarrow Q$ función de transducción

#### Ejemplo: Parsing con expresiones regulares

Con fines de prácticidad vamos a _imitar_ el comportamiento de un transductor utilizando el modulo de python `re`

La estructura del sustantivo en español es:

` BASE+AFIJOS (marcas flexivas)   --> Base+DIM+GEN+NUM`

In [ ]:
palabras = [
    "niño",
    "niños",
    "niñas",
    "niñitos",
    "gato",
    "gatos",
    "gatitos",
    "perritos",
    "paloma",
    "palomita",
    "palomas",
    "flores",
    "flor",
    "florecita",
    "lápiz",
    "lápices",
    #"chiquitititititos",
    #'curriculum', # curricula
    #'campus', # campi
]

In [ ]:
def morph_parser_rules(words: list[str]) -> list[str]:
    """Aplica reglas morfológicas a una lista de palabras para realizar
    un análisis morfológico.

    Parameters:
    ----------
    words : list of str
        Lista de palabras a las que se les aplicarán las reglas morfológicas.

    Returns:
    -------
    list of str
        Una lista de palabras después de aplicar las reglas morfológicas.
    """

    # Lista para guardar las palabras parseadas
    morph_parsing = []

    # Reglas que capturan ciertos morfemas
    # {ecita, itos, as, os}
    for w in words:
        # ecit -> DIM
        R0 = re.sub(r"([^ ]+)ecit([a|o|as|os])", r"\1-DIM\2", w)
        # it -> DIM
        R1 = re.sub(r"([^ ]+)it([a|o|as|os])", r"\1-DIM\2", R0)
        # a(s) -> FEM
        R2 = re.sub(r"([^ ]+)a(s)", r"\1-FEM\2", R1)
        # a -> FEM
        R3 = re.sub(r"([^ ]+)a\b", r"\1-FEM", R2)
        # o(s) -> MSC
        R4 = re.sub(r"([^ ]+)o(s)", r"\1-MSC\2", R3)
        # o .> MSC
        R5 = re.sub(r"([^ ]+)o\b", r"\1-MSC", R4)
        # es -> PL
        R6 = re.sub(r"([^ ]+)es\b", r"\1-PL", R5)
        # s -> PL
        R7 = re.sub(r"([^ ]+)s\b", r"\1-PL", R6)
        # Sustituye la c por z cuando es necesario
        parse = re.sub(r"c-", r"z-", R7)

        # Guarda los parseos
        morph_parsing.append(parse)
    return morph_parsing

In [ ]:
def prettify_tags(word: str) -> str:
    tags = {
        "DIM": "[b bright_yellow]DIM[/]",
        "FEM": "[b green3]FEM[/]",
        "MSC": "[b medium_purple1]MSC[/]",
        "PL": "[b deep_sky_blue1]PL[/]",
    }
    for tag, pretty_tag in tags.items():
        word = word.replace(tag, pretty_tag)
    return word

In [ ]:
morph_parsing = morph_parser_rules(palabras)
for palabra, parseo in zip(palabras, morph_parsing):
    rprint(palabra, "-->", prettify_tags(parseo))

#### Preguntas 🤔
- ¿Qué pasa con las reglas en lenguas donde son más comunes los prefijos y no los sufijos?
- ¿Cómo podríamos identificar características de las lenguas?

#### Herramientas para hacer sistemas de análisis morfológico basados en reglas

- [Apertium](https://en.wikipedia.org/wiki/Apertium)
- [Foma](https://github.com/mhulden/foma/tree/master)
- [Helsinki Finite-State Technology](https://hfst.github.io/)
- Ejemplo [proyecto](https://github.com/apertium/apertium-yua) de analizador morfológico de Maya Yucateco
- Ejemplo normalizador ortográfico del [Náhuatl](https://github.com/ElotlMX/py-elotl/tree/master)


También se pueden utilizar diferentes métodos de aprendizaje de máquina para realizar análisis/generación morfológica. En los últimos años ha habido un shared task de [morphological reinflection](https://github.com/sigmorphon/2023InflectionST) para poner a competir diferentes métodos

### Segmentación morfológica

#### Corpus: [SIGMORPHON 2022 Shared Task on Morpheme Segmentation](https://github.com/sigmorphon/2022SegmentationST/tree/main)

- Shared task donde se buscaba convertir las palabras en una secuencia de morfemas
- Dividido en dos partes:
    - Segmentación a nivel de palabras ☀️
    - Segmentación a nivel oraciones

#### Track: words

| word class | Description                      | English example (input ==> output)     |
|------------|----------------------------------|----------------------------------------|
| 100        | Inflection only                  | played ==> play @@ed                   |
| 010        | Derivation only                  | player ==> play @@er                   |
| 101        | Inflection and Compound          | wheelbands ==> wheel @@band @@s        |
| 000        | Root words                       | progress ==> progress                  |
| 011        | Derivation and Compound          | tankbuster ==> tank @@bust @@er        |
| 110        | Inflection and Derivation        | urbanizes ==> urban @@ize @@s          |
| 001        | Compound only                    | hotpot ==> hot @@pot                   |
| 111        | Inflection, Derivation, Compound | trackworkers ==> track @@work @@er @@s

#### Explorando el corpus

In [ ]:
response = r.get("https://raw.githubusercontent.com/sigmorphon/2022SegmentationST/main/data/spa.word.test.gold.tsv")

In [ ]:
LANGS = {
    "ces": "Czech",
    "eng": "English",
    "fra": "French",
    "hun": "Hungarian",
    "spa": "Spanish",
    "ita": "Italian",
    "lat": "Latin",
    "rus": "Russian",
}
CATEGORIES = {
    "100": "Inflection",
    "010": "Derivation",
    "101": "Inflection, Compound",
    "000": "Root",
    "011": "Derivation, Compound",
    "110": "Inflection, Derivation",
    "001": "Compound",
    "111": "Inflection, Derivation, Compound"
}

In [ ]:
def get_track_files(lang: str, track: str = "word") -> list[str]:
    """Genera una lista de nombres de archivo del shared task

    Con base en el idioma y el track obtiene el nombre de los archivos
    para con información reelevante para hacer análisis estadístico.
    Esto es archivos .test y .dev

    Parameters:
    ----------
    lang : str
        Idioma para el cual se generarán los nombres de archivo.
    track : str, optional
        Track del shared task de donde vienen los datos (por defecto es "word").

    Returns:
    -------
    list[str]
        Una lista de nombres de archivo generados para el idioma y la pista especificados.
    """
    return [
        f"{lang}.{track}.test.gold",
        f"{lang}.{track}.dev",
    ]

In [ ]:
def get_raw_corpus(files: list) -> list:
    """Descarga y concatena los datos de los archivos tsv desde una URL base.

    Parameters:
    ----------
    files : list
        Lista de nombres de archivos (sin extensión) que se descargarán
        y concatenarán.

    Returns:
    -------
    list
        Una lista que contiene los contenidos descargados y concatenados
        de los archivos tsv.
    """
    result = []
    for file in files:
        print(f"Downloading {file}.tsv", end=" ")
        response = r.get(f"https://raw.githubusercontent.com/sigmorphon/2022SegmentationST/main/data/{file}.tsv")
        print(f"status={response.status_code}")
        lines = response.text.split("\n")
        result.extend(lines[:-1])
    return result

In [ ]:
def raw_corpus_to_dataframe(corpus_list: list, lang: str) -> pd.DataFrame:
    """Convierte una lista de datos de corpus en un DataFrame

    Parameters:
    ----------
    corpus_list : list
        Lista de líneas del corpus a convertir en DataFrame.
    lang : str
        Idioma al que pertenecen los datos del corpus.

    Returns:
    -------
    pd.DataFrame
        Un DataFrame de pandas que contiene los datos del corpus procesados.
    """
    data_list = []
    for line in corpus_list:
        try:
            word, tagged_data, category = line.split("\t")
        except ValueError:
            # Caso donde no existe la categoria
            word, tagged_data = line.split("\t")
            category = "NOT_FOUND"
        morphemes = tagged_data.split()
        data_list.append(
            {"words": word, "morph": morphemes, "category": category, "lang": lang}
        )
    df = pd.DataFrame(data_list)
    df["word_len"] = df["words"].apply(lambda x: len(x))
    df["morph_count"] = df["morph"].apply(lambda x: len(x))
    return df

In [ ]:
files = get_track_files("spa")
raw_spa = get_raw_corpus(files)
df = raw_corpus_to_dataframe(raw_spa, lang="spa")

In [ ]:
df.head(20)

#### Análisis cuantitativo para el Español

In [ ]:
def plot_histogram(df, kind, lang):
    """Genera un histograma de frecuencia para una columna específica
    en un DataFrame.

    Parameters:
    ----------
    df : pd.DataFrame
        DataFrame que contiene los datos para generar el histograma.
    kind : str
        Nombre de la columna para la cual se generará el histograma.
    lang : str
        Idioma asociado a los datos.

    Returns:
    -------
    None
        Esta función muestra el histograma usando matplotlib.
    """
    counts = df[kind].value_counts().head(30)
    plt.bar(counts.index, counts.values)
    plt.xlabel(kind)
    plt.ylabel('Frequency')
    plt.title(f'{kind} Frequency Graph for {lang}')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_histogram(df, "category", "spanish")

#### Ejemplo 🥑: Adivina, adivinador, probabilidad del tipo de morfemas

- No todos los afijos son iguales. Algunos sirven para conjugar (Flexivos) y otros para crear conceptos nuevos (Derivativos).
- Aprovechando que el dataset de SIGMORPHON nos dice si una palabra es de tipo `Inflection` (100) o `Derivation` (010) calculemos qué tan probable es que un morfema sea flexivo o derivativo.


In [ ]:
def analyze_morpheme_types(df: pd.DataFrame) -> pd.DataFrame:
    """Calcula y obtiene las probabilidades de que un morfema sea flexivo o derivativo.
    """
    inflection_counts = defaultdict(int)
    derivation_counts = defaultdict(int)
    total_counts = defaultdict(int)

    for _, row in df.iterrows():
        morphemes = row['morph'] 
        category = row['category']

        # Con base en SIGMORPHON:
        # Pos 0: Inflection, Pos 1: Derivation
        is_inflection = category[0] == '1'
        is_derivation = category[1] == '1'

        for morph in morphemes:
            clean_morph = morph.replace("@@", "")

            # Ignoramos morfemas muy cortos o vacíos
            if len(clean_morph) < 2:
                continue

            total_counts[clean_morph] += 1
            if is_inflection:
                inflection_counts[clean_morph] += 1
            if is_derivation:
                derivation_counts[clean_morph] += 1

    results = []
    for morph, total in total_counts.items():
        if total > 5:
            p_inflection = inflection_counts[morph] / total
            p_derivation = derivation_counts[morph] / total
            results.append({
                'morfema': morph,
                'total_freq': total,
                'prob_flexion': round(p_inflection, 3),
                'prob_derivacion': round(p_derivation, 3)
            })

    return pd.DataFrame(results).sort_values(by='total_freq', ascending=False)

In [ ]:
morph_stats = analyze_morpheme_types(df)

#### Los morfemas más "Gramaticales" (Flexivos)

Deberíamos ver terminaciones verbales y plurales.

In [ ]:
rprint("Top Morfemas Flexivos:")
rprint(morph_stats.sort_values(by=['prob_flexion', "total_freq"], ascending=False).head(10))

#### Los morfemas más "Léxicos" (Derivativos)

Deberíamos ver sufijos que cambian sustantivos a adjetivos, etc.

In [ ]:
rprint("Top Morfemas Derivativos:")
rprint(morph_stats.sort_values(by=['prob_derivacion', "total_freq"], ascending=False).head(10))

#### Morfosintaxis

- Etiquetas que hacen explícita la funcion gramatical de las palabras en una oración
- Determina la función de la palabra dentro la oración (por ello se le llama Partes del Discurso)
- Se le conoce tambien como **Análisis morfosintáctico**: es el puente entre la estructura de las palabras y la sintaxis
- Permiten el desarrollo de herramientas de NLP más avanzadas
- El etiquetado es una tarea que se puede abordar con técnicas secuenciales, por ejemplo, HMMs, CRFs, Redes neuronales

<center><img src="https://byteiota.com/wp-content/uploads/2021/01/POS-Tagging.jpg" height=500 width=500></center

#### Ejemplo

> El gato negro rie malvadamente

- El - DET
- gato - NOUN
- negro - ADJ
- ríe - VER

<center><img src="https://i.pinimg.com/originals/0e/f1/30/0ef130b255ea704625b2ad473701dee5.gif"></center

### Materia prima de otras tareas de NLP

- Named entity recognition (NER)
- Statistical language models
- Text generation
- Sentient analysis

## Etiquetado automático POS (*POS tagging*)

- El etiquetado POS es una tarea del NLP dónde se le asigna de forma automática una etiqueta según su función a cada palabra tomando en cuenta el contexto de la oración.

- En esta tarea se toma en cuenta cierta estructura de la oración.

- En un enfoque probabilístico queremos obtener: $P(\overrightarrow{x},\overrightarrow{y})$


donde:
- $\overrightarrow{x}$ = $<x_1,x_2,...,x_n>$
- $\overrightarrow{y}$ = $<y_1,y_2,...,y_n>$
- $x_i = palabra$ y $y_i = etiqueta\ POS$

### Un primer acercamiento: Hidden Markov Models (HMM)

<center><img src="https://www.davidsbatista.net/assets/images/2017-11-11-HMM.png"></center>

$p(\overrightarrow{x},\overrightarrow{y}) = \displaystyle\prod_{i=1}^{n} p(y_i|y_{i-1}) ⋅ p(x_i|y_i)$

Donde:
- $\overrightarrow{y} = secuencia\ de\ etiquetas\ POS$
- $\overrightarrow{x} = secuencia\ de\ palabras$

### Suposición de Markov

> "The probability of a particular state is dependent only on the previous state"

#### Características

- Clasificador secuencial
    - Dada una secuencia de entrada se predice la secuencia de salida más probable
    - Se apreden los parámetros de secuencias previamente etiquetadas

### Componentes del los *HMM*


- Estados (etiquetas): $T = t_1,t_2,...,t_n$
- Observaciones (palabras): $W = w_1,w_2,...,w_n$
- Estados iniciales y finales

#### Probabilidades asociadas a estados

- Matriz $A$ con las probabilidades de ir de un estado a otro
- Matriz $B$ con las probabilidades de que una observasión se genera a partir de un estado
- Probabilidades asociadas a los estados iniciales y finales

### ¿Qué soluciona HMM?

1. Aprender parámetros asociados con una secuencia de observación dada (training step)
    - Dada una lista de palabras y sus etiquetas POS asociadas, el modelo aprende la estructura dada
2. Aplicar un modelo HMM previamente entrenado
    - Dada una nueva oración nunca antes vista, se puede **predecir** la etiqueta POS asociada a cada palabra de dicha oración usando la estructura aprendida

### Corpus: `cess_esp`

Corpus con 1M de palabras etiquetadas para Español y Catalán - https://www.nltk.org/book/ch02.html#tab-corpora


In [ ]:
# Descargando el corpus cess_esp
nltk.download('cess_esp')

In [ ]:
# Cargando oraciones
corpora = cess_esp.tagged_sents()

In [ ]:
corpora[1][:5]

In [ ]:
def get_tags_map() -> dict:
    """sauce https://gist.github.com/vitojph/39c52c709a9aff2d1d24588aba7f8155/
    """
    tags_raw = r.get(
        "https://gist.githubusercontent.com/vitojph/39c52c709a9aff2d1d24588aba7f8155/raw/af2d83bc4c2a7e2e6dbb01bd0a10a23a3a21a551/universal_tagset-ES.map"
    ).text.split("\n")
    tags_map = {line.split("\t")[0].lower(): line.split("\t")[1] for line in tags_raw}
    return tags_map


def map_tag(tag: str, tags_map=get_tags_map()) -> str:
    if tags_map.get(tag.lower()) == ".":
        return "PUNCT"
    return tags_map.get(tag.lower(), "N/F")


def parse_tags(corpora: list[list[tuple]]) -> list[list[tuple]]:
    result = []
    for sentence in corpora:
        result.append([(word, map_tag(tag)) for word, tag in sentence])
    return result

In [ ]:
corpora = parse_tags(corpora)

In [ ]:
corpora[0]

In [ ]:
len(corpora)

### Implementación de HMMs

In [ ]:
# Separando en dos conjuntos, uno para entrenamiento y otro para pruebas
train_data, test_data = train_test_split(corpora, test_size=0.3, random_state=42)

In [ ]:
# Comprobemos la longitud de la data
len(train_data), len(test_data)

In [ ]:
assert len(train_data) + len(test_data) == len(corpora), "Something is wrong with the split :("

#### Entrenamiento

In [ ]:
from nltk.tag import hmm

# Creando el modelo HMM usando nltk
trainer = hmm.HiddenMarkovModelTrainer()

# Hora de entrenar
hmm_model = trainer.train(train_data)

#### Resultados

In [ ]:
tagged_test_data = hmm_model.tag_sents([[word for word, _ in sent] for sent in test_data])

In [ ]:
tagged_test_data[0]

In [ ]:
# Extrayendo tags verdaderas vs tags predichas
y_true = [tag for sent in test_data for _, tag in sent]
y_pred = [tag for sent in tagged_test_data for _, tag in sent]

In [ ]:
y_true[:3]

In [ ]:
y_pred[:3]

In [ ]:
def report_accuracy(y_true: list, y_pred: list) -> defaultdict:
    """Construye un reporte de exactitud por etiqueta."""
    label_accuracy_counts = defaultdict(lambda: {"correct": 0, "total": 0})

    for gold_tag, predicted_tag in zip(y_true, y_pred):
        label_accuracy_counts[gold_tag]["total"] += 1
        if gold_tag == predicted_tag:
            label_accuracy_counts[gold_tag]["correct"] += 1
    
    # Calculate and display the accuracy for each label
    print("Label\tAccuracy")
    for label, counts in label_accuracy_counts.items():
        accuracy = counts["correct"] / counts["total"] if counts["total"] > 0 else 0.0
        print(f"{label}\t{accuracy * 100:.2f}%")
    return label_accuracy_counts

In [ ]:
label_accuracy_counts = report_accuracy(y_true, y_pred)

### Hablemos de Métricas

#### Confusion Matrix (binaria)

Es una forma tabular de vizualizar el desempeño de un modelo de *Machine Learning (ML)*. En las columnas tenemos la cuenta de etiquetas predichas mientras que en las filas tenemos la cuenta de las etiquetas reales (o viceversa)

![](https://i1.wp.com/dataaspirant.com/wp-content/uploads/2020/08/3_confusion_matrix.png?ssl=1)

- **TP:** Etiquetas correctamente predichas como positivas.
    - Ej: Se etiqueto un correo como spam y era spam
- **FP:** Etiquetas incorrectamente predichas. 
    - Ej: Se etiqueto un correo como spam y NO era spam
- **TN:** Etiquetas correctamente predichas como negativas.
    - Ej: Se etiqueto un correo como no spam y era no spam
- **FN:** Etiquetas incorrectamente predichas como negativas.
    - Ej: Se etiqueto un correo como no spam y era spam

#### Accuracy = $\frac{TP + TN}{TP + TN + FP + FN}$

Es una de las métricas más sencillas usadas en *ML*. Define que tan exacto es el modelo. Por ejemplo, si de 100 etiquetas el modelo acerto en 90 tendremos un accuracy de 0.90 o 90%

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred, y_true))

#### Precision = $\frac{TP}{TP + FP}$

Indica la relación entre las predicciones positivas correctas (SPAM es SPAM) con el total de predicciones de la clase sin importar si fueron correctas o no (Todo lo que fue marcado como SPAM correctamente con todo lo que fue marcado como SPAM incorrectamente). *De los correos etiquetados como SPAM cuandos fueron efectivamente SPAM*

In [ ]:
from sklearn.metrics import precision_score
print(precision_score(y_pred, y_true, average="macro"))

#### Recall = $\frac{TP}{TP + FN}$

Indica la relacion entre las predicciones positivas correctas con el total de predicciones incorrectas de otras clases (Todo lo que no se marco como SPAM cuando si era SPAM). *Todos los correos que en realidad eran SPAM*

In [ ]:
from sklearn.metrics import recall_score
print(recall_score(y_pred, y_true, average="macro"))

#### F1-score = $\frac{2PR}{P + R}$

Es el promedio ponderado entre *precision* y *recall*. Toma en cuenta tanto los FP como los FN.

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_pred, y_true, average="macro"))

#### Un ejemplo concreto

In [ ]:
nltk.download('punkt_tab')

In [ ]:
unseen_sentence = "La casa es grande y luminosa."
#unseen_sentence = "La muchacha vio al dinosaurio con el telescopio"
#unseen_sentence = "El gato rie malvadamente"

# Tokenizando
tokenized_sentence = nltk.word_tokenize(unseen_sentence)

# Haciendo predicciones
predicted_tags = [tag for word, tag in hmm_model.tag(tokenized_sentence)]

print("Palabra \tPOS Tag (predicha)")
for word, tag in zip(tokenized_sentence, predicted_tags):
    print(f"{word}\t{tag}")

#### Comparando con modelos pre-entrenados

- [Model information](https://spacy.io/models/es)

In [ ]:
!python -m spacy download es_core_news_sm

In [ ]:
import spacy
# spacy.cli.download("") # Direct from python

nlp = spacy.load("es_core_news_sm")
doc = nlp(unseen_sentence)
print([(w.text, w.pos_) for w in doc])

In [ ]:
from spacy import displacy
displacy.render(doc, style="dep", jupyter=True)

In [ ]:
test_sentences = [" ".join([word for word, _ in sent]) for sent in test_data][:10]

In [ ]:
docs = [nlp(sent) for sent in test_sentences]

In [ ]:
for doc in docs[:1]:
    print()
    print([(w.text, w.pos_) for w in doc])

In [ ]:
tagged_test_data[0]

### ¿Limitaciones?

- Cada estado depende exclusivamente de su predecesor inmediato
    - Dependencias limitadas
- Cada observación depende exclusivamente del estado actual
- Probabilidades estáticas
    - Ejemplo, si veo un par de tags `(V) -> (S)` no importa si esta al inicio, en medio o al final de la oración la probabilidad siempre será la misma

![](https://3.bp.blogspot.com/-pPGGqs462yw/T1ol64kj9uI/AAAAAAAAAKI/CDCiH1IJodE/w1200-h630-p-k-nu/patricio.jpg)

## Sobrepasando las fronteras: _Conditional Random Fields (CRFs)_

- Modelo de gráficas **no dirigido**. Generaliza los *HMM*
    - Adiós a la *Markov assuption*
    - Podemos tener cualquier dependencia que queramos entre nodos
    - Nos enfocaremos en un tipo en concreto: *LinearChain-CRFs* ¡¿Por?!

<center><img width=300 src="https://i.kym-cdn.com/entries/icons/original/000/032/676/Unlimited_Power_Banner.jpg"></center>


- Modela la probabilidad **condicional** $P(Y|X)$
    - Modelo discriminativo
    - Probabilidad de un estado oculto dada **toda** la secuecia de entrada
![homer](https://media.tenor.com/ul0qAKNUm2kAAAAd/hiding-meme.gif)

- Captura mayor **número de dependencias** entre las palabras y captura más características
    - Estas se definen en las *feature functions* 🙀
- El entrenamiento se realiza aplicando gradiente decendente y optimización con algoritmos como [L-BFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS)


<center><img src="https://iameo.github.io/images/gradient-descent-400.gif"></center>


$P(\overrightarrow{y}|\overrightarrow{x}) = \frac{1}{Z} \displaystyle\prod_{i=1}^N exp\{w^T ⋅ \phi(y_{i-1}, y_i, \overrightarrow{x}, i)\}$

Donde:
- $\overrightarrow{y}$ = Etiquetas POS
- $\overrightarrow{x}$ = Palabras en una oración
- $w^T$ = Vector de pesos a aprender
- $\phi$ = Vector de *Features*
    - Calculado con base en un conjunto de *feature functions*
- $i$ = la posición actual en la oración
- $Z$ = factor de normalización

![](https://aman.ai/primers/ai/assets/conditional-random-fields/Conditional_Random_Fields.png)

Tomado de http://www.davidsbatista.net/blog/2017/11/13/Conditional_Random_Fields/

#### Feature functions

$\phi(y_{i-1}, y_i, \overrightarrow{x}, i)$

- Parte fundamental de los CRFs
- Cuatro argumentos:
    - Todos los datos observables $\overrightarrow{x}$ (conectar $x$ con cualquier $y$)
    - El estado oculto anterior $y_{i-1}$
    - El estado oculto actual $y_i$
    - El index del timestamp $i$
        - Cada feature list puede tener diferentes formas

- Aquí es donde está la flexibilidad del modelo
- Tantas *features* como queramos, las que consideremos que pueden ayudar a que el modelo tenga un mejor desempeño
    - Íntimamente ligadas a la lengua. Para mejor desempeño se debe hacer un análisis de sus características.
- Ejemplo:

```python
[
    "word.lower()",
    "EOS",
    "BOS",
    "pre-word",
    "nxt-word",
    "word-position",
    ...
]
```

### Implementación de CRFs

#### Feature lists

In [ ]:
def word_to_features(sent, i):
    word = sent[i][0]
    features = {
        "word.lower()": word.lower(),
        "word[-3:]": word[-3:],
        "word[-2:]": word[-2:],
    }
    if i > 0:
        prev_word = sent[i - 1][0]
        features.update(
            {
                "prev_word.lower()": prev_word.lower(),
            }
        )
    else:
        # Beginning of sentence
        features["BOS"] = True
    return features


# Extract features and labels
def sent_to_features(sent) -> list:
    return [word_to_features(sent, i) for i in range(len(sent))]


def sent_to_labels(sent) -> list:
    return [label for token, label in sent]

In [ ]:
len(corpora)

In [ ]:
corpora[0]

In [ ]:
# Preparando datos para el CRF
X = [sent_to_features(sent) for sent in corpora]
y = [sent_to_labels(sent) for sent in corpora]

In [ ]:
# Exploración de data estructurada
print(X[0][0])
print(len(X[0]))

In [ ]:
print(y[0][0])
print(len(y[0]))

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
assert len(X_train) + len(X_test) == len(corpora), "Something wrong with my split :("
assert len(y_train) + len(y_test) == len(corpora), "Something wrong with my split :("

#### Entrenamiento

In [ ]:
# Initialize and train the CRF tagger: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html
crf = CRF(
    algorithm="lbfgs",
    c2=0.001,
    max_iterations=10,
    verbose=True,
)
try:
    crf.fit(X_train, y_train)
except AttributeError as e:
    print(e)

#### Evaluación

In [ ]:
y_pred = crf.predict(X_test)

# Flatten the true and predicted labels
y_test_flat = [label for sent_labels in y_test for label in sent_labels]
y_pred_flat = [label for sent_labels in y_pred for label in sent_labels]

# Evaluate the model
report = classification_report(y_true=y_test_flat, y_pred=y_pred_flat)
rprint(report)

#### Ejercicio 🗺️: Experimentación y prueba del *CRF*

- Experimentación
    - Agrega más características a la función `word_to_features()`
        - ¿Qué características pueden ser útiles?
    - Experimenta con diferentes hiperparámetros del CRF
    - En ambos casos observa cómo afectan su rendimiento
- Prueba
    - Usando el mejor modelo aprendido por el CRF etiqueta una oración desafiante
    - Imprime el resultado
    - ¿Las etiquetas son corretas? 

# Tarea 1: Exploración de Niveles del lenguaje 🔭

### FECHA DE ENTREGA: 10 de Marzo 2026 at 11:59pm

### Fonética

1. Con base en el sistema de búsqueda visto en la [práctica 1](https://github.com/umoqnier/cl-2026-2-lab/blob/main/notebooks/1_niveles_linguisticos_I.ipynb), dónde se recibe una palabra ortográfica y devuelve sus transcripciones fonológicas, proponga una solución para los casos en que la palabra buscada no se encuentra en el lexicón/diccionario.
    - ¿Cómo devolver o **aproximar** su transcripción fonológica?
    - Reutiliza el sistema de búsqueda visto en clase y mejóralo con esta funcionalidad.
    - Muestra al menos tres ejemplos

### Morfología

2. Elige tres lenguas del corpus que pertenezcan a familias lingüísticas distintas
   - Ejemplo: `spa` (Romance), `eng` (Germánica), `hun` (Urálica)
   - Para cada una de las tres lenguas calcula y compara:
       -  **Ratio morfemas / palabra**: El promedio de morfemas que componen las palabras
        -  **Indicé de Flexión / Derivación**: Del total de morfemas, ¿Qué porcentaje son etiquetas de flexión (`100`) y cuáles de derivación (`010`)?
3. Visualización
    - Genera una figura con **subplots** para comparar las lenguas lado a lado.
    - *Plot 1*: Distribución de la longitud de los morfemas
    - *Plot 2*: Distribución de las categorías (flexión, derivación, raíz, etc.)
4. Con base en esta información, responde la pregunta: *¿Cuál de las tres lenguas se comporta más como una lengua aglutinante y cuál como una lengua aislante?*
    - Justifica tu respuesta usando tus métricas y figuras

### EXTRA:

- Genera la [matriz de confusión](https://en.wikipedia.org/wiki/Confusion_matrix) para el etiquetador CRFs visto en clase
- Observando las etiquetas donde el modelo falló responde las preguntas:
    - ¿Por qué crees que se confundió?
    - ¿Es un problema de ambigüedad léxica (la palabra tiene múltiples etiquetas)?
    - ¿Qué *features* añadirías para solucionarlo?